In [1]:
%set_env CMAKE_ARGS="-DLLAMA_CUBLAS=on"  
%pip install -U openai sentence-transformers tokenizers python-dotenv llama-cpp-python

env: CMAKE_ARGS="-DLLAMA_CUBLAS=on"
  Obtaining dependency information for tokenizers from https://files.pythonhosted.org/packages/57/bd/45b5ef6b088880779f70acf60027f7043ca5fa1b98f4a4345cf3aea09044/tokenizers-0.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached tokenizers-0.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import openai
from dotenv import load_dotenv
from llama_cpp import Llama
import json

load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
openai.api_key = OPENAI_API_KEY

GGUF_MODEL_PATH = "./model/nous-hermes-llama2-13b-q4.gguf"
MAX_CONTEXT = 4096
# GGUF_MODEL_PATH = "./model/nous-hermes-llama-2-7b-q4.gguf"
llm = Llama(model_path=GGUF_MODEL_PATH, n_gpu_layers=999, n_ctx=MAX_CONTEXT, verbose=False)


ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3090, compute capability 8.6
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from ./model/nous-hermes-llama2-13b-q4.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  5120, 32032,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q5_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_K     [  5120, 13824,     1,     

In [2]:
def get_response_from_model(prompt):
        return llm(
                prompt, 
                max_tokens=MAX_CONTEXT, 
                # stop=["\n"], 
                echo=False, 
                temperature=0.5
               )["choices"][0]["text"]


In [11]:
def get_information_description_prompt(information):
    example_information = "Monster: Kobold\nTrait: Sunlight Sensitivity\nWhile in sunlight, the kobold has disadvantage on attack rolls, as well as on Wisdom (Perception) checks that rely on sight."
    example_result = "The negative effects of sunlight on kobolds."
    example_information2 = "Monster: Aerosaur\nAerosaurs include several varieties of large reptiles with leathery wings, including pterodons and sunwings."
    example_result2 = "Background information about Aerosaurs and subspecies."
    return f"""
### Instruction:
You are a helpful assistant that assists with indexing information. Describe the information that is contained in the following text. Make sure to include important context.
Include any important details that might not be explicitly stated in the text. The information is about Dungeons and Dragons.
Be concise. Answer in a single sentence.

### Input:
{example_information}

### Response:
{example_result}

### Input:
{example_information2}

### Response:
{example_result2}

### Input:
{information}

### Response:

            """

def get_information_retrieval_prompt(query):
    example_query = "How does sunlight effect kobolds?"
    example_result = "Effects of sunlight on kobolds."
    example_query2 = "What is an aerosaur?"
    example_result2 = "Description of an aerosaur."

    return f"""
### Instruction:
You are a helpful assistant that assists with retrieving information. Describe the information that is required to answer the following query relating to monsters.
Do not answer the following query, just describe the information that would be required to answer the query. Do not provide an answer, only a description. Answer in a single sentence.
Be concise.

### Input:
{example_query}

### Response:
{example_result}

### Input:
{example_query2}

### Response:
{example_result2}

### Input:
{query}

### Response:

            """

info = "Hold Breath\nWhile out of water, the octopus can hold its breath for 1 hour."
prompt = get_information_description_prompt(info)
response = get_response_from_model(prompt)
print(response)

prompt = get_information_retrieval_prompt("How long can octopuses hold their breath out of water?")
response = get_response_from_model(prompt)
print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   262.94 ms
llama_print_timings:      sample time =     7.28 ms /    18 runs   (    0.40 ms per token,  2471.51 tokens per second)
llama_print_timings: prompt eval time =   154.26 ms /   109 tokens (    1.42 ms per token,   706.59 tokens per second)
llama_print_timings:        eval time =   234.78 ms /    17 runs   (   13.81 ms per token,    72.41 tokens per second)
llama_print_timings:       total time =   417.39 ms
Llama.generate: prefix-match hit


 The ability of an octopus to stay underwater for an extended period of time.
 Time octopuses can stay out of water and hold their breath.



llama_print_timings:        load time =   262.94 ms
llama_print_timings:      sample time =     6.06 ms /    15 runs   (    0.40 ms per token,  2475.25 tokens per second)
llama_print_timings: prompt eval time =   191.93 ms /   154 tokens (    1.25 ms per token,   802.36 tokens per second)
llama_print_timings:        eval time =   189.37 ms /    14 runs   (   13.53 ms per token,    73.93 tokens per second)
llama_print_timings:       total time =   404.04 ms


In [12]:
enriched_data = []

with open('./monster_text.json') as f:
    monster_texts = json.load(f)

for monster_text in monster_texts:
    info = ""
    if monster_text.get("monster_name") is not None:
        info += f"Monster: {monster_text.get('monster_name')}\n"
    if monster_text.get("field") is not None:
        info += f"{monster_text.get('field')}: {monster_text.get('trait_name')}\n"
    if monster_text.get("text") is not None:
        info += monster_text.get("text")
    prompt = get_information_description_prompt(info)
    response = get_response_from_model(prompt)
    monster_text["information_description"] = response
    
    print(monster_text["information_description"])

    enriched_data.append(monster_text)
    if len(enriched_data) % 100 == 0:
        print(f"Saving data at {len(enriched_data)}")
        with open('./monster_text_enriched.json', 'w') as f:
            json.dump(enriched_data, f)

 The unusual dietary habits of the Crystal Greatwyrm.



llama_print_timings:        load time =   262.94 ms
llama_print_timings:      sample time =     6.47 ms /    16 runs   (    0.40 ms per token,  2472.19 tokens per second)
llama_print_timings: prompt eval time =   126.70 ms /    27 tokens (    4.69 ms per token,   213.11 tokens per second)
llama_print_timings:        eval time =   208.38 ms /    15 runs   (   13.89 ms per token,    71.98 tokens per second)
llama_print_timings:       total time =   360.24 ms
Llama.generate: prefix-match hit


 Attack patterns of the Crystal Greatwyrm.



llama_print_timings:        load time =   262.94 ms
llama_print_timings:      sample time =     4.79 ms /    12 runs   (    0.40 ms per token,  2503.13 tokens per second)
llama_print_timings: prompt eval time =   128.34 ms /    31 tokens (    4.14 ms per token,   241.55 tokens per second)
llama_print_timings:        eval time =   152.66 ms /    11 runs   (   13.88 ms per token,    72.06 tokens per second)
llama_print_timings:       total time =   299.43 ms
Llama.generate: prefix-match hit


 The Crystal Greatwyrm's attack and damage statistics for its bite action.


...

 The Draconic Shard's attack options and special ability.



llama_print_timings:        load time =   262.94 ms
llama_print_timings:      sample time =     6.06 ms /    15 runs   (    0.40 ms per token,  2473.61 tokens per second)
llama_print_timings: prompt eval time =   126.11 ms /    28 tokens (    4.50 ms per token,   222.02 tokens per second)
llama_print_timings:        eval time =   194.35 ms /    14 runs   (   13.88 ms per token,    72.03 tokens per second)
llama_print_timings:       total time =   343.44 ms
Llama.generate: prefix-match hit


 The Draconic Shard's telekinetic attack that deals force damage to a single target within reach or at range.



llama_print_timings:        load time =   262.94 ms
llama_print_timings:      sample time =    11.21 ms /    28 runs   (    0.40 ms per token,  2496.66 tokens per second)
llama_print_timings: prompt eval time =   140.57 ms /    70 tokens (    2.01 ms per token,   497.96 tokens per second)
llama_print_timings:        eval time =   380.13 ms /    27 runs   (   14.08 ms per token,    71.03 tokens per second)
llama_print_timings:       total time =   563.71 ms


ValueError: Requested tokens (541) exceed context window of 512